# Visualisation - du notebook à la webapp

L'objectif de ce TD est de :
- Créer son premier widget
- Ajouter des éléments d'interactivité à un graphique Plotly
- transfomer le notebook de visualisation interactive en webapp (application web) avec Voilà
- déployer et partager cette application web avec ngrok ou binder

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 1) Création d'un premier widget

a) Exécutez le code ci-dessous pour importer la librairie et la renommer widgets. On utilsera

In [ ]:
import ipywidgets as widgets
from IPython.display import display

b) La liste complète des widgets disponible est disponible à [cette adresse](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html).

A l'aide de cette documentation, construisez une variable `my_int_slider` qui contiendra un widget "IntSlider" avec 5 comme valeur par défaut, 1 en minimum et 20 en maximum. On pourra laisser step égal à 1 comme niveau d'incrément.

Utilisez la fonction "display" pour l'afficher.
        

In [ ]:
my_int_slider = widgets.IntSlider(value=5,
                                  min=1,
                                  max=20,
                                  step=1)

display(my_int_slider)

d) A quoi corresponnd le résultat de cette opération ?

In [ ]:
my_int_slider.keys

e) En déduire comment afficher la valeur du slider.

Faites varier la valeur du slider et vérifier qu'en exécutant de nouveau la ligne de code, la valeur a bien changé.

In [ ]:
print(my_int_slider.value)

# 2) Création d'un User Form

Les widgets peuvent servir à créer des user forms pour des utilisateurs finaux.

Cet exercice a pour but de construire un outil qui permet de sélectionner de sélectionner un indice boursier pour afficher un scatter plot avec l'indice d'apple AAPL.

a) Importez les librairies suivantes

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interactive_output
import io

b) Dans une variable `my_upload_widget`, créez un widget de type Fileupload qui accepte l'extension .csv et un seul fichier à la fois. Puis testez l'import de ce fichier [csv exemple](https://drive.google.com/file/d/1mci2f7Jn4mqjS75Yvv73c6-bsraPutDj/view?usp=sharing).

In [ ]:
my_upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description="Uploadez votre csv")
display(my_upload_widget)

c) Le données en sortie sont un peu difficile à extraire.

In [ ]:
df = pd.read_csv(io.BytesIO(my_upload_widget.value[0].content))
df

d) Représentez dans un graphique Plotly Scatter les donnnées de GOOG versus APPL avec une courbe de tendance (trendlines='ols') pour voir la corrélation entre ces deux indices.

In [ ]:
px.scatter(df, x="GOOG", y="AAPL", trendline="ols").show()

e) Créez une fonction `financial_scatter_widget` qui prendra en entrée les paramètres df, x et y, elle affichera le graphique créé précédemment pour les noms de colonnes x et y.

In [ ]:
def financial_scatter_widget(df, x=None, y=None):
    px.scatter(df, x=x, y=y, trendline="ols").show()

financial_scatter_widget(df, x="GOOG", y="AAPL")

f) L'idée est de donner la main à l'utilisateur pour choisir quel indice il souhaite vérifier la correlation avec l'indice APPL.

Créez un widget de type Dropdown, qui permettra de sélectionner le nom d'une colonne du DataFrame créé.

In [ ]:
df.set_index('date', inplace=True)
df

In [ ]:
x_dropdown = widgets.Dropdown(
    options=df.columns.drop("AAPL"),
    value='GOOG',
    description='AAPL vs ',
    style= {'description_width': 'initial'},
    disabled=False,
)
display(x_dropdown)

g) A l'aide de la fonction `interact()` de widgets, faites exécutez la fonction `financial_scatter_widget` en fonction de la colonne choisie.

On fixera les paramètres `df`=Le DataFrame récupéré de l'upload  et `y`="APPL" à l'aide de la fonction `fixed()`.

In [ ]:
widgets.interact(
    financial_scatter_widget,
    df=fixed(df),
    y=fixed('AAPL'),
    x=x_dropdown
)

On aurait aussi pu faire ça
```python
def on_change(change):
    global df
    if change['type'] == 'change' and change['name'] == 'value':
        financial_scatter_widget(df, x=change['new'], y='AAPL')

x_dropdown.observe(on_change)
```

# 3) Transformation du notebook en application web avec Voilà

Voici un code python permettant de créer un graphe aléatoire :

In [ ]:
from ipywidgets import interact
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx

def random_lobster(n, m, k, p):
    return nx.random_lobster(n, p, p / m)

def powerlaw_cluster(n, m, k, p):
    return nx.powerlaw_cluster_graph(n, m, p)

def erdos_renyi(n, m, k, p):
    return nx.erdos_renyi_graph(n, p)

def newman_watts_strogatz(n, m, k, p):
    return nx.newman_watts_strogatz_graph(n, k, p)

def plot_random_graph(n, m, k, p, generator):
    g = generator(n, m, k, p)
    nx.draw(g)
    plt.show()

interact(plot_random_graph, n=(2,30), m=(1,10), k=(1,10), p=(0.0, 0.99, 0.001),
         generator=[
             ('lobster', random_lobster),
             ('power law', powerlaw_cluster),
             ('Newman-Watts-Strogatz', newman_watts_strogatz),
             (u'Erdős-Rényi', erdos_renyi),
         ]);

Copier coller ce code dans un nouveau notebook 'my_webapp.ipynb' et depuis un terminal, lancez la commande :
```
voila my_webapp.ipynb
```

# [Bonus] Déploiement d'une webapp avec Binder

Pour pouvoir utiliser [Binder](https://jupyter.org/binder), il faut au préalable avoir pushé son notebook dans un repo git et avoir rajouté un fichier 'requirements.txt' contenant les librairies nécessaires pour faire tourner votre projet.

Ensuite, il suffit d'aller sur https://mybinder.org/ et :
- dans 'GitHub', copier coller l'URL de votre projet git
- dans 'Path to a notebook file (optional)', écrivez 'voila/render/ce_notebook.ipynb' en précisant bien 'URL' au lieu de 'File'. Celà permet de spécifier à Binder qu'il faut directement utiliser Voila pour créer la webapp.

Cliquez ensuite sur 'lauch'. Une image docker de votre projet sera créer grace au fichier 'requirements.txt'. Vous aurez ensuite un lien pour lancer un conteneur docker autonome qui vous permettra d'acceder à votre application web. Pour chaque utilisateur utilisant ce lien, un conteneur docker sera créé.